<a href="https://colab.research.google.com/github/ppijbb/Keras_jupyter/blob/main/Convolutional_Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [ ]:
import tensorflow as tf
import numpy as np
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.astype(np.float32).reshape(-1,28*28)/255.0
x_test = x_test.astype(np.float32).reshape(-1,28*28)/255.0
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

y_train = y_train.astype(np.int32)
y_test = y_train.astype(np.int32)

tf.convert_to_tensor(x_train).get_shape()

x = tf.placeholder("float", [None, 784]) 
y_=tf.placeholder('float',[None,10])
x_image = tf.reshape(x, [-1,28,28,1])

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32]) 

In [ ]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x): 
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], 
padding='SAME')
    
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5, 5, 32, 64]) 
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2) 
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([7 * 7 * 64, 1024]) 
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64]) 
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [ ]:
W_fc2 = weight_variable([1024, 10]) 
b_fc2 = bias_variable([10])
y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy) 
sess = tf.Session()
sess.run(tf.initialize_all_variables()) 

# 2만 번의 학습
for i in range(20000):
    batch = mnist.train.next_batch(50)
    sess.run(train_step,feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

In [ ]:
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1)) 
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
sess = tf.Session()
sess.run(tf.initialize_all_variables())
for i in range(20000):
    batch = mnist.train.next_batch(50)
    if i%1000 == 0:
        train_accuracy = sess.run( accuracy, feed_dict={
        x:batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy)) 
sess.run(train_step,feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

step 0, training accuracy 0.12
step 1000, training accuracy 0.1
step 2000, training accuracy 0.14
step 3000, training accuracy 0.1
step 4000, training accuracy 0.08
step 5000, training accuracy 0.14
step 6000, training accuracy 0.08
step 7000, training accuracy 0.08
step 8000, training accuracy 0.16
step 9000, training accuracy 0.06
step 10000, training accuracy 0.16
step 11000, training accuracy 0.08
step 12000, training accuracy 0.08
step 13000, training accuracy 0.1
step 14000, training accuracy 0.14
step 15000, training accuracy 0.06
step 16000, training accuracy 0.2
step 17000, training accuracy 0.08
step 18000, training accuracy 0.08
step 19000, training accuracy 0.12


In [ ]:
print("test accuracy %g"% sess.run(
        accuracy, feed_dict={x: x_test, y_: y_test, keep_prob: 1.0}))

InvalidArgumentError: ignored